In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gsm8k-dataset/gsm8k/README.md
/kaggle/input/gsm8k-dataset/gsm8k/socratic/train-00000-of-00001.parquet
/kaggle/input/gsm8k-dataset/gsm8k/socratic/test-00000-of-00001.parquet
/kaggle/input/gsm8k-dataset/gsm8k/main/train-00000-of-00001.parquet
/kaggle/input/gsm8k-dataset/gsm8k/main/test-00000-of-00001.parquet


In [ ]:
!pip install perplexityai
!pip install perplexity

In [ ]:
pip install pyarrow

In [3]:
import pyarrow.parquet as pa

In [5]:
table = pa.read_table('/kaggle/input/gsm8k-dataset/gsm8k/main/train-00000-of-00001.parquet')
df = table.to_pandas()

In [7]:
work = []
answer = []
for i in range(0, len(df)):
    text = df["answer"][i]
    cleaned_text = re.sub(r"<<.*?>>", "", text)
    cleaned_text = cleaned_text.split("####")[0].strip()
    match = re.search(r"####\s*(.*)", text)
    work.append(cleaned_text)
    answer.append(int(match.group(1).replace(",", "")))

In [8]:
df = df.drop(columns=['answer'])
df['work'] = work
df['answer'] = answer

In [ ]:
# file_path = "modified_gsm8k.csv"
# df.to_csv(file_path, index=False)

In [9]:
# from openai import OpenAI
from perplexity import Perplexity

In [10]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("Api_key")

In [12]:
# client = Perplexity()

# Or specify explicitly
client = Perplexity(api_key=api)

In [ ]:
# import requests

# endpoint = "https://api.perplexity.ai/chat/completions"
# headers = {"Authorization": f"Bearer {api}", "Content-Type": "application/json"}
# data = {"model": "sonar", "messages": [{"role": "user", "content": "Who is the prime minister of India"}]}
# response = requests.post(endpoint, headers=headers, json=data)
# print(response.text.choices[0].message.content)

In [154]:
initial_prompt = "Please carefully read the problem and restate it clearly, identifying all relevant quantities, units, and relationships. Then, break down the problem into smaller, manageable parts and solve each step logically and sequentially. Use precise proportional reasoning and convert any percentages or ratios into concrete numbers based on total quantities. Maintain consistent units throughout, performing explicit conversions when necessary. Apply logical deduction to validate assumptions and rule out impossible scenarios. At each step, explain your reasoning before proceeding. Verify intermediate results to catch errors early. Finally, provide a clear, concise answer that directly addresses the question."

In [42]:
import json

In [43]:
def str_to_dict(response: str) -> dict:
    fixed_string = response.replace("'", "\'")
    data_dict = json.loads(fixed_string)
    return data_dict

In [131]:
def talk_to_llm(system: str, user: str, r_format = {}) -> dict:
    completion = client.chat.completions.create(
        model = "sonar",
        messages = [
            {"role":"system","content": system},
            {"role":"user", "content":user}
        ],
        temperature = 0.2,
        response_format = r_format
    )
    return str_to_dict(completion.choices[0].message.content)

In [59]:
def generate_response(prompt, question):
    system = """ Response must be a JSON object with exactly two keys:
    "works" → the detailed reasoning, explanation, or supporting steps.
    "answer" → The final numerical result."""
    user = (
        f"My instruction:\n<prompt>{prompt}</prompt>\n"
        f"Question:\n{question}\n"
    )
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "schema": {
                "type": "object",
                "properties": {
                    "work": {"type": "string"},
                    "answer": {"type": "integer"}
                }
            }
        }
    }
    out = talk_to_llm(system, user,response_format)
    return out

In [60]:
def check_response(pred, actual):
    if pred == actual:
        return 1
    else:
        return 0

In [92]:
def positive_reflection(initial_prompt, examples, num):
    system = "output JSON: {\"key_points\": [string, ...]}."
    user = (
        "As a logician, you are good at breaking down the internal logic of the problem step by step.\n"
        f"<prompt>{initial_prompt}</prompt>\n"
        f"<examples>{json.dumps(examples, ensure_ascii=False)}</examples>\n"
        "I have provided you with triples of question, generated answer, and actual answer. "
        f"Summarize the {num} most valuable key points to improve accuracy. "
    )
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "schema": {
                "type": "object",
                "properties": {
                    "key_points": {"type": "string"}
                }
            }
        }
    }
    out = talk_to_llm(system, user, response_format)
    # out["key_points"] = (out.get("key_points") or [])[:num]
    return out

In [103]:
def negative_reflection(initial_prompt, examples, num):
    system = "output JSON: {\"Reasons\": [string, ...]}."
    user = (
        "As a logician, you are good at breaking down the internal logic of the problem step by step.\n"
        f"<prompt>{initial_prompt}</prompt>\n"
        f"<examples>{json.dumps(examples, ensure_ascii=False)}</examples>\n"
        "I have provided you with triples of question, wrong answer, and actual answer. "
        f"Identify {num} primary reasons why the prompt causes these wrong answers. "
    )
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "schema": {
                "type": "object",
                "properties": {
                    "Reasons": {"type": "string"}
                }
            }
        }
    }
    out = talk_to_llm(system, user,response_format)
    # out["reasons"] = (out.get("reasons") or [])[:num]
    return out

In [135]:
def strategy_decider(initial_prompt: str, example_blob: str, experience_text: str):
    system = "You are an expert prompt engineer. Output JSON: {\"strategy\": string, \"steps\": [string,... <=10]}."
    demo = """
    <demo>
    <prompt>read the given paragraph and identify the most logical answer among the options.</prompt>
    <example>
    question: The following paragraphs each describe a set of five objects arranged in a fixed order. The statements are logically consistent within each paragraph. In a golf tournament, there were five golfers: Eve, Amy, Ada, Rob, and Joe. Amy finished second-to-last. Rob finished below Eve. Ada finished above Joe. Joe finished second.
    Options:
    (A) Eve finished last
    (B) Amy finished last
    (C) Ada finished last
    (D) Rob finished last
    (E) Joe finished last
    Answer: (B) Amy finished last
    Target: (D) Rob finished last
    </example>
    <experience> One primary reason mistakes occur in this task is due to misunderstanding or misinterpretation of the logical order and relationships presented in the paragraphs</experience>
    <strategy>
    Here is a strategy guide how to achieve "understanding or interpretation of the logical order and relationships":
    1. Carefully read the entire paragraph to understand the context and the objects or individuals involved.
    2. Identify the logical relationships or orderings described in the paragraph.
    3. Create a visual aid such as a list or a diagram. Place the objects or individuals from left to right based
    on the logical relationships. The leftmost object or individual would be the first in the order and the
    rightmost would be the last.
    4. As you read each relationship, adjust the positions of the objects or individuals in your visual aid
    accordingly.
    5. Once all relationships have been considered, your visual aid should represent the correct order of the
    objects or individuals.
    </strategy>
    </demo>
    """
    user = f""" As an expert in prompt engineer, your task is to create a step-by-step strategy guide on how to use specific experience based on provided prompt.
    # Begin Demos
    {demo}
    # End Demos
    My current prompt is:
    <prompt>{initial_prompt}</prompt>
    And here is the task data:
    <example>{example_blob}</example>
    Through comprehensive analysis of the data, I’ve gained an experience that can improve the prompt:
    <experience>{experience_text}</experience>
    Based on my current prompt, please generate a strategy to address the above experience.
    The strategy is:
    """
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "schema": {
                "type": "object",
                "properties": {
                    "strategy": {"type": "string"},
                    "steps":{"type": "string"}
                }
            }
        }
    }
    out = talk_to_llm(system, user, response_format)
    # out["steps"] = (out.get("steps") or [])[:10]
    return out


In [151]:
def prompt_rewriter(initial_prompt: str, example_blob: str, experience_text: str, strategy_text: str) -> str:
    system = "Rewrite the instruction. Output JSON: {\"improved_prompt\": string}."
    user = f"""
    My current instruction is:
    <prompt>{initial_prompt}</prompt>
    And Here are some task data:
    <example>{example_blob}</example>
    Through comprehensive analysis, I get an experience and corresponding strategy:
    # Experience
    <experience>{experience_text}</experience>
    # Strategy
    <strategy>{strategy_text}</strategy>
    Based on my current prompt, refer to this experience and the strategy, write 1 different improved prompt.
    """
    response_format = {
        "type": "json_schema",
        "json_schema": {
            "schema": {
                "type": "object",
                "properties": {
                    "improved_prompt":{"type": "string"}
                }
            }
        }
    }
    out = talk_to_llm(system, user,response_format)
    return out

In [155]:
batch_df = df.sample(n=200, random_state=39)

In [156]:
batch_df.head(1)

,question,work,answer
2318,It takes a dog groomer 10 minutes to dry a sho...,It takes 10 minutes to dry the coat of a short...,4


In [157]:
corrects, wrongs = [],[]

In [158]:
i = 1
for ex in batch_df.itertuples(index=False):
    # print(ex.question)
    pred = generate_response(initial_prompt, ex.question)
    check = check_response(pred["answer"], ex.answer)
    record = {"question": ex.question, "generated_answer": pred["work"],"actual_answer": ex.work}
    (corrects if check else wrongs).append(record)
    print(i,"done")
    i = i+1

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123 done
124 done
125 done

In [159]:
len(wrongs)

9

In [160]:
wrongs

[{'question': 'There is a massive rainstorm lasting 4 days.  There is an area that collects water to prevent flooding in the area.  It ends up overflowing on the 4th day.  The area can hold the equivalent of 6 feet of rain.  It can also drain out the equivalent of 3 inches of rain per day to the nearby river without causing problems.  The first day it rained 10 inches.  The second day it rained twice that much.  On the third day, it rained 50% more than the second day.  It flooded the fourth day before getting a chance to do any of the draining.  What is the minimum amount it rained on the fourth day?',
  'generated_answer': 'The problem describes a rainstorm lasting 4 days with an area that collects rainwater to prevent flooding. The area can hold up to 6 feet of rainwater equivalent and can drain 3 inches of rain per day to a nearby river without causing problems. The rain amounts for the first three days are given, and the area flooded on the 4th day before any draining occurred. We

In [85]:
wrongs

[{'question': 'There is a massive rainstorm lasting 4 days.  There is an area that collects water to prevent flooding in the area.  It ends up overflowing on the 4th day.  The area can hold the equivalent of 6 feet of rain.  It can also drain out the equivalent of 3 inches of rain per day to the nearby river without causing problems.  The first day it rained 10 inches.  The second day it rained twice that much.  On the third day, it rained 50% more than the second day.  It flooded the fourth day before getting a chance to do any of the draining.  What is the minimum amount it rained on the fourth day?',
  'generated_answer': 'The area can hold 6 feet of rain total. Convert 6 feet to inches: 6 feet × 12 inches/foot = 72 inches total capacity.\n\nThe area drains 3 inches per day without problems.\n\nRainfall per day:\n- Day 1: 10 inches\n- Day 2: twice Day 1 = 2 × 10 = 20 inches\n- Day 3: 50% more than Day 2 = 20 + 0.5 × 20 = 30 inches\n\nCalculate the net water stored after each day con

In [91]:
num_reflect = 5

In [94]:
positive = positive_reflection(initial_prompt, corrects, num_reflect) if corrects else {"key_points":[]}

In [95]:
positive

{'key_points': '1. Carefully define and restate the problem to identify all relevant quantities and relationships before attempting to solve it, ensuring a clear understanding of what is being asked[2][5][6].\n2. Break down complex problems into smaller, manageable parts or sub-problems, solving each step logically and sequentially to build towards the final solution[2][5].\n3. Use proportional reasoning and ratios accurately, converting percentages and ratios into concrete numbers by applying them to total quantities, and verify calculations step-by-step to avoid errors[1][5].\n4. Apply consistent units and conversions (e.g., minutes to hours, pounds to ounces) and carefully track units throughout calculations to maintain accuracy and clarity[1][5].\n5. Employ logical deduction and elimination strategies, such as if-then statements and process of elimination, to rule out impossible scenarios and confirm valid conclusions in logic problems[1][3].'}

In [104]:
negative = negative_reflection(initial_prompt, wrongs, num_reflect) if wrongs else {"reasons":[]}

In [119]:
negative = {"reasons": "1. Misinterpretation or misapplication of problem data leading to erroneous assumptions. \n 2. Inconsistent or incorrect use of units and failure to convert measurements properly.\n 3. Improper logical structuring and step-by-step reasoning causing miscalculations.\n 4. Failure to distinguish between similar but distinct quantities (e.g., totals versus parts, current versus future values).\n 5. Lack of clear problem goal identification and explicit definition of what is being solved for"}

In [124]:
exp_text = ""

# Ensure key_points is a list
key_points = positive.get("key_points")
if key_points:
    if isinstance(key_points, str):
        key_points = [key_points]  # wrap single string in a list
    exp_text += "Strengths to amplify: " + "; ".join(key_points) + ". "

# Ensure reasons is a list
reasons = negative.get("reasons")
if reasons:
    if isinstance(reasons, str):
        reasons = [reasons]
    exp_text += "Failure modes to fix: " + "; ".join(reasons) + "."


In [125]:
exp_text

'Strengths to amplify: 1. Carefully define and restate the problem to identify all relevant quantities and relationships before attempting to solve it, ensuring a clear understanding of what is being asked[2][5][6].\n2. Break down complex problems into smaller, manageable parts or sub-problems, solving each step logically and sequentially to build towards the final solution[2][5].\n3. Use proportional reasoning and ratios accurately, converting percentages and ratios into concrete numbers by applying them to total quantities, and verify calculations step-by-step to avoid errors[1][5].\n4. Apply consistent units and conversions (e.g., minutes to hours, pounds to ounces) and carefully track units throughout calculations to maintain accuracy and clarity[1][5].\n5. Employ logical deduction and elimination strategies, such as if-then statements and process of elimination, to rule out impossible scenarios and confirm valid conclusions in logic problems[1][3].. \nFailure modes to fix: 1. Misi

In [126]:
example_blob = json.dumps((corrects + wrongs), ensure_ascii=False)

In [136]:
strat = strategy_decider(initial_prompt, example_blob, exp_text)

In [145]:
strat["strategy"]

'Enhance prompt clarity and structure to guide the model through precise, stepwise reasoning, ensuring accurate interpretation and calculation.'

In [148]:
strategy = f"Strategy is : {strat['strategy']}\n Steps to perform strategy : {strat['steps']}"

In [149]:
strategy

"Strategy is : Enhance prompt clarity and structure to guide the model through precise, stepwise reasoning, ensuring accurate interpretation and calculation.\n Steps to perform strategy : 1. Begin by explicitly restating the problem to identify all relevant quantities, units, and relationships, clarifying what is being asked.\n2. Break down complex problems into smaller, manageable parts or sub-problems, addressing each sequentially.\n3. Use precise proportional reasoning and ratio calculations, converting percentages and ratios into concrete numbers based on total quantities.\n4. Maintain consistent units throughout calculations, performing necessary conversions explicitly and tracking units carefully.\n5. Apply logical deduction and elimination strategies to validate assumptions and rule out impossible scenarios.\n6. Clearly define the goal or target quantity at the outset and revisit it after each step to ensure alignment.\n7. Incorporate chain-of-thought prompting by instructing th

In [152]:
r1 = prompt_rewriter(initial_prompt, example_blob, exp_text, strategy)

In [153]:
r1

{'improved_prompt': 'Please carefully read the problem and restate it clearly, identifying all relevant quantities, units, and relationships. Then, break down the problem into smaller, manageable parts and solve each step logically and sequentially. Use precise proportional reasoning and convert any percentages or ratios into concrete numbers based on total quantities. Maintain consistent units throughout, performing explicit conversions when necessary. Apply logical deduction to validate assumptions and rule out impossible scenarios. At each step, explain your reasoning before proceeding. Verify intermediate results to catch errors early. Finally, provide a clear, concise answer that directly addresses the question.'}

In [133]:
completion = client.chat.completions.create(
  model="sonar",
  messages=[
      {"role": "system", "content": "output JSON: {\"Reasons\": [string, ...]}."},
      {"role":"user", "content": "As a logician, you are good at breaking down the internal logic of the problem step by step.\n"
        f"<prompt>{initial_prompt}</prompt>\n"
        f"<examples>{json.dumps(wrongs, ensure_ascii=False)}</examples>\n"
        "I have provided you with triples of question, wrong answer, and actual answer. "
        "Identify 5 primary reasons why the prompt causes these wrong answers."
      }
  ]
)

i


In [134]:
print(completion.choices[0].message.content)

The 5 primary reasons why the prompt causes these wrong answers are:

1. **Lack of Explicit Instruction to Verify Units and Consistency**  
   The wrong answers often stem from failing to convert or align units properly (e.g., mixing feet and inches, confusing number of symbols with number of cards) or misunderstanding the base quantities involved. The prompt does not emphasize systematic unit conversion or verifying assumptions about the problem’s parameters, leading to errors in the basics of problem setup.

2. **Inadequate Emphasis on Breaking Down Problems into Clear, Logical Steps**  
   The examples show mistakes made because the prompt does not require the solver to explicitly outline or verify the internal logical chain step-by-step, such as identifying what is known, what needs to be found, and how intermediate sub-results relate. Without enforcing a detailed breakdown of the problem, solvers jump to conclusions or skip needed intermediate reasoning.

3. **Ambiguity and Insuff